In [1]:
# -*- coding: utf-8 -*-
import sys
sys.path.append('C:/Users/pc/Documents/GitHub/Private_Project/personal_project')
from collections import defaultdict
from datetime import datetime, timedelta
import time
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
import pickle
import chat_bot as cb
import Database_Handler as dh
from multiprocessing import Pool
from functools import partial
import pickle
import urllib3
import json
def OpenAPI():
    import pickle
    openapi = pickle.load(open('./etri_open_api_access_key', 'rb'))
    return openapi
def USE_ETRI_AI(analysisCode, text):    
    openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU"
    accessKey = OpenAPI().strip()
    requestJson = {
        'access_key' : accessKey,
        'argument' : {
            'text' : text,
            'analysis_code' : analysisCode
        }
    }
    http = urllib3.PoolManager()
    response = http.request(
        "POST",
        openApiURL,
        headers={"Content-Type": "application/json; charset=UTF-8"},
        body=json.dumps(requestJson)
    )
    return json.loads(str(response.data,'utf-8'))

if __name__ == '__main__':
    pass

In [2]:
mongodb = dh.ToMongoDB(*dh.AWS_MongoDB_Information())
dbname = 'hy_db'
useDb = dh.Use_Database(mongodb, dbname)
slack = cb.Slacker(cb.slacktoken())
useCollection = dh.Use_Collection(useDb, 'newsDaum')

In [3]:
x = useCollection.find_one({'category':"뉴스"})

In [4]:
x['mainText']

" 주민들은 공장에서 배출된 연기가 '남성이 매우 건강해지는 부작용'을 일으킨다며, 이러한 부작용은 사람에 그치지 않고 개에도 나타난다고 지적했다. 이어 부작용으로 '일상생활에 어려움을 느낄 정도'라며 역학조사가 필요하다고 덧붙였다. 지역에서 간호사로 일하는 한 여성은 “공장 인근을 산책하던 중 강아지가 흥분상태인 것을 확인했다”며 “공장이 환경규제를 준수하는지 확인해야 한다”고 주장했다. 공장은 약 20년 전 마을에 들어선 후 현재까지 치료제를 생산했는데 최근 들어 지역 남성들에게 예상치 못한 영향을 미치고 있다. 시민들 문제 제기에 공장 대변인은 “사랑의 연기는 존재하지 않는다”고 일축하며 “남성들에게서 ‘플라세보효과‘가 나타난 듯하다”고 밝혔다. 한편 공장 성명에도 ‘알 수 없는 큰 힘이 작용한다‘는 생각에 반려견과 이곳을 찾는 남성들이 부쩍 늘었다고 전해진다. 이런 모습에 “건강은 연기가 아닌 운동에서 비롯된 것”이라는 추측이 나오고 있다.  * 플라세보효과는 의사가 효과 없는 가짜 약 또는 꾸며낸 치료법을 환자에게 처방하고, 환자의 긍정적인 믿음으로 인해 병세가 호전되는 현상을 말한다. 이동준 기자 blondie@segye.com\n사진= 선데이타임스 캡처"

In [5]:
from konlpy.tag import Mecab, Twitter
from konlpy.utils import pprint
import nltk

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

In [11]:
mecab_tagger = Mecab()
twitter_tagger = Twitter()

In [12]:
mecab_tagger.pos(x['mainText'])

[('주민', 'NNG'),
 ('들', 'XSN'),
 ('은', 'JX'),
 ('공장', 'NNG'),
 ('에서', 'JKB'),
 ('배출', 'NNG'),
 ('된', 'XSV+ETM'),
 ('연기', 'NNG'),
 ('가', 'JKS'),
 ("'", 'SY'),
 ('남성', 'NNG'),
 ('이', 'JKS'),
 ('매우', 'MAG'),
 ('건강', 'NNG'),
 ('해', 'XSV+EC'),
 ('지', 'VX'),
 ('는', 'ETM'),
 ('부작용', 'NNG'),
 ("'", 'SY'),
 ('을', 'JKO'),
 ('일으킨다', 'VV+EC'),
 ('며', 'EC'),
 (',', 'SC'),
 ('이러', 'XR'),
 ('한', 'XSA+ETM'),
 ('부작용', 'NNG'),
 ('은', 'JX'),
 ('사람', 'NNG'),
 ('에', 'JKB'),
 ('그치', 'VV'),
 ('지', 'EC'),
 ('않', 'VX'),
 ('고', 'EC'),
 ('개', 'NNG'),
 ('에', 'JKB'),
 ('도', 'JX'),
 ('나타난다고', 'VV+EC'),
 ('지적', 'NNG'),
 ('했', 'XSV+EP'),
 ('다', 'EF'),
 ('.', 'SF'),
 ('이어', 'MAG'),
 ('부작용', 'NNG'),
 ('으로', 'JKB'),
 ("'", 'SY'),
 ('일상', 'NNG'),
 ('생활', 'NNG'),
 ('에', 'JKB'),
 ('어려움', 'NNG'),
 ('을', 'JKO'),
 ('느낄', 'VV+ETM'),
 ('정도', 'NNG'),
 ("'", 'SY'),
 ('라며', 'VCP+EC'),
 ('역학', 'NNG'),
 ('조사', 'NNG'),
 ('가', 'JKS'),
 ('필요', 'NNG'),
 ('하', 'XSV'),
 ('다고', 'EC'),
 ('덧붙였', 'VV+EP'),
 ('다', 'EF'),
 ('.', 'SF'),
 ('지역', '

In [14]:
twitter_tagger.pos(x['mainText'])

[('주민', 'Noun'),
 ('들', 'Suffix'),
 ('은', 'Josa'),
 ('공장', 'Noun'),
 ('에서', 'Josa'),
 ('배출', 'Noun'),
 ('된', 'Verb'),
 ('연기', 'Noun'),
 ('가', 'Josa'),
 ("'", 'Punctuation'),
 ('남성', 'Noun'),
 ('이', 'Josa'),
 ('매우', 'Noun'),
 ('건강해', 'Adjective'),
 ('지는', 'Eomi'),
 ('부작용', 'Noun'),
 ("'", 'Punctuation'),
 ('을', 'Josa'),
 ('일으킨', 'Verb'),
 ('다며', 'Eomi'),
 (',', 'Punctuation'),
 ('이러한', 'Adjective'),
 ('부작용', 'Noun'),
 ('은', 'Josa'),
 ('사람', 'Noun'),
 ('에', 'Josa'),
 ('그치', 'Verb'),
 ('지', 'Eomi'),
 ('않고', 'Verb'),
 ('개', 'Noun'),
 ('에도', 'Josa'),
 ('나타난', 'Verb'),
 ('다고', 'Eomi'),
 ('지적했', 'Verb'),
 ('다', 'Eomi'),
 ('.', 'Punctuation'),
 ('이어', 'Verb'),
 ('부작용', 'Noun'),
 ('으로', 'Josa'),
 ("'", 'Punctuation'),
 ('일상생활', 'Noun'),
 ('에', 'Josa'),
 ('어려움', 'Noun'),
 ('을', 'Josa'),
 ('느낄', 'Verb'),
 ('정도', 'Noun'),
 ("'", 'Punctuation'),
 ('라며', 'Noun'),
 ('역학', 'Noun'),
 ('조사', 'Noun'),
 ('가', 'Josa'),
 ('필요하다', 'Adjective'),
 ('고', 'Eomi'),
 ('덧붙였', 'Verb'),
 ('다', 'Eomi'),
 ('.', 'Punctu

In [17]:
# 형태소 분석 : morp
# 어휘의미 분석  (동음이의어) : wsd
# 어휘의미 분석 (다의어 분석) : wsd_poly
# 개체명 인식  : ner
# 의존 구문 분석  : dparse
# 의미역 인식  : srl
test = USE_ETRI_AI('srl', x['mainText'])

In [18]:
test2 = test['return_object']
test3 = test2['sentence']

In [19]:
morp = [] ; wsd = [] ; word = [] ; ne = [] ; dependency = []
for i in test3:
    raw_text = i['text'] 
    morp += list(map(lambda x: (x['lemma'], x['type']),i['morp']))
    wsd += list(map(lambda x: (x['text'], x['type']),i['WSD']))
    word += list(map(lambda x: (x['text'], x['type']),i['word']))
    ne += list(map(lambda x: (x['text'], x['type']),i['NE']))
    dependency = list(map(lambda x: (x['text'], x['label']),i['dependency']))
    srl = i['SRL']

In [20]:
morp

[('주민', 'NNG'),
 ('들', 'XSN'),
 ('은', 'JX'),
 ('공장', 'NNG'),
 ('에서', 'JKB'),
 ('배출', 'NNG'),
 ('되', 'XSV'),
 ('ㄴ', 'ETM'),
 ('연기', 'NNG'),
 ('가', 'JKS'),
 ("'", 'SS'),
 ('남성', 'NNG'),
 ('이', 'JKS'),
 ('매우', 'MAG'),
 ('건강', 'NNG'),
 ('하', 'XSA'),
 ('어', 'EC'),
 ('지', 'VX'),
 ('는', 'ETM'),
 ('부', 'XPN'),
 ('작용', 'NNG'),
 ("'", 'SS'),
 ('을', 'JKO'),
 ('일으키', 'VV'),
 ('ㄴ', 'EC'),
 ('다며', 'EC'),
 (',', 'SP'),
 ('이러하', 'VA'),
 ('ㄴ', 'ETM'),
 ('부', 'XPN'),
 ('작용', 'NNG'),
 ('은', 'JX'),
 ('사람', 'NNG'),
 ('에', 'JKB'),
 ('그치', 'VV'),
 ('지', 'EC'),
 ('않', 'VX'),
 ('고', 'EC'),
 ('개', 'NNG'),
 ('에', 'JKB'),
 ('도', 'JX'),
 ('나타나', 'VV'),
 ('ㄴ다고', 'EC'),
 ('지적', 'NNG'),
 ('하', 'XSV'),
 ('었', 'EP'),
 ('다', 'EF'),
 ('.', 'SF'),
 ('잇', 'VV'),
 ('어', 'EC'),
 ('부', 'XPN'),
 ('작용', 'NNG'),
 ('으로', 'JKB'),
 ("'", 'SS'),
 ('일상', 'NNG'),
 ('생활', 'NNG'),
 ('에', 'JKB'),
 ('어려움', 'NNG'),
 ('을', 'JKO'),
 ('느끼', 'VV'),
 ('ㄹ', 'ETM'),
 ('정도', 'NNG'),
 ("'", 'SS'),
 ('이', 'VCP'),
 ('라며', 'EC'),
 ('역학', 'NNG'),
 ('조사

In [101]:
twit_param_grid = {'tfidfvectorizer__min_df' : [3, 5, 7], 
                  'tfidfvectorizer__ngram_range' : [(1,1), (1,2), (1,3)], 
                   'logisticregression__C':[0.1, 1, 10]}

In [102]:
twit_pipe = make_pipeline(TfidfVectorizer(tokenizer = twitter_tokenizer), LogisticRegression())

In [103]:
twit_grid = GridSearchCV(twit_pipe, twit_param_grid)

In [ ]:
twit_grid.fit(df2, y)
print ("최상의 교차 검증 점수 : {}".format(twit_grid.best_score_))
print ('최적의 교차 검증 매개변수 : {}'.format(twit_grid.best_params_))

In [92]:
def tokenize(doc):
    return ['/'.join(t) for t in pos_tagger.pos(doc, norm=True, stem=True)]

In [93]:
t = [(tokenize(row[1]), row[2]) for row in x]

KeyboardInterrupt: 

In [ ]:

pprint(t[0])

In [42]:
# 형태소 분석 : morp
# 어휘의미 분석  (동음이의어) : wsd
# 어휘의미 분석 (다의어 분석) : wsd_poly
# 개체명 인식  : ner
# 의존 구문 분석  : dparse
# 의미역 인식  : srl
x = USE_ETRI_AI('srl', x['mainText'])

In [44]:
xx = x['return_object']

In [52]:
xxx = xx['sentence']

In [57]:
for i in xxx:
    raw_text = i['text'] 
    morp = list(map(lambda x: (x['lemma'], x['type']),i['morp']))
    wsd = list(map(lambda x: (x['text'], x['type']),i['wsd']))
    word = i['word']
    ne = i['NE']
    dependency = i['dependency']
    srl = i['SRL']
    break

In [71]:
srl

[{'argument': [{'text': '공장',
    'type': 'ARG2',
    'weight': 0.120684,
    'word_id': 1.0},
   {'text': '연기', 'type': 'ARG1', 'weight': 0.274273, 'word_id': 3.0}],
  'sense': 1.0,
  'verb': '배출',
  'weight': 0.197479,
  'word_id': 2.0},
 {'argument': [{'text': "'남성",
    'type': 'ARG1',
    'weight': 0.142192,
    'word_id': 4.0},
   {'text': '매우', 'type': 'ARGM-EXT', 'weight': 0.48364, 'word_id': 5.0}],
  'sense': 1.0,
  'verb': '건강',
  'weight': 0.312916,
  'word_id': 6.0},
 {'argument': [{'text': '주민들',
    'type': 'ARG0',
    'weight': 0.326694,
    'word_id': 0.0},
   {'text': '연기', 'type': 'ARG0', 'weight': 0.381284, 'word_id': 3.0},
   {'text': "'남성이 매우 건강해지는 부작용'",
    'type': 'ARG1',
    'weight': 0.482167,
    'word_id': 7.0}],
  'sense': 1.0,
  'verb': '일으키',
  'weight': 0.396715,
  'word_id': 8.0},
 {'argument': [{'text': '이러한 부작용',
    'type': 'ARG1',
    'weight': 0.524875,
    'word_id': 10.0}],
  'sense': 1.0,
  'verb': '이러하',
  'weight': 0.524875,
  'word_id': 9.0},

In [ ]:
i.ke

In [58]:
raw_text

" 주민들은 공장에서 배출된 연기가 '남성이 매우 건강해지는 부작용'을 일으킨다며, 이러한 부작용은 사람에 그치지 않고 개에도 나타난다고 지적했다."

In [62]:
morp

[{'id': 0.0, 'lemma': '주민', 'position': 1.0, 'type': 'NNG', 'weight': 0.9},
 {'id': 1.0,
  'lemma': '들',
  'position': 7.0,
  'type': 'XSN',
  'weight': 0.0299841},
 {'id': 2.0, 'lemma': '은', 'position': 10.0, 'type': 'JX', 'weight': 0.100393},
 {'id': 3.0, 'lemma': '공장', 'position': 14.0, 'type': 'NNG', 'weight': 0.9},
 {'id': 4.0,
  'lemma': '에서',
  'position': 20.0,
  'type': 'JKB',
  'weight': 0.153407},
 {'id': 5.0, 'lemma': '배출', 'position': 27.0, 'type': 'NNG', 'weight': 0.9},
 {'id': 6.0,
  'lemma': '되',
  'position': 33.0,
  'type': 'XSV',
  'weight': 0.000224177},
 {'id': 7.0,
  'lemma': 'ㄴ',
  'position': 33.0,
  'type': 'ETM',
  'weight': 0.392321},
 {'id': 8.0,
  'lemma': '연기',
  'position': 37.0,
  'type': 'NNG',
  'weight': 0.652338},
 {'id': 9.0,
  'lemma': '가',
  'position': 43.0,
  'type': 'JKS',
  'weight': 0.066324},
 {'id': 10.0, 'lemma': "'", 'position': 47.0, 'type': 'SS', 'weight': 1.0},
 {'id': 11.0,
  'lemma': '남성',
  'position': 48.0,
  'type': 'NNG',
  'weig

[('주민', 'NNG'),
 ('들', 'XSN'),
 ('은', 'JX'),
 ('공장', 'NNG'),
 ('에서', 'JKB'),
 ('배출', 'NNG'),
 ('되', 'XSV'),
 ('ㄴ', 'ETM'),
 ('연기', 'NNG'),
 ('가', 'JKS'),
 ("'", 'SS'),
 ('남성', 'NNG'),
 ('이', 'JKS'),
 ('매우', 'MAG'),
 ('건강', 'NNG'),
 ('하', 'XSA'),
 ('어', 'EC'),
 ('지', 'VX'),
 ('는', 'ETM'),
 ('부', 'XPN'),
 ('작용', 'NNG'),
 ("'", 'SS'),
 ('을', 'JKO'),
 ('일으키', 'VV'),
 ('ㄴ', 'EC'),
 ('다며', 'EC'),
 (',', 'SP'),
 ('이러하', 'VA'),
 ('ㄴ', 'ETM'),
 ('부', 'XPN'),
 ('작용', 'NNG'),
 ('은', 'JX'),
 ('사람', 'NNG'),
 ('에', 'JKB'),
 ('그치', 'VV'),
 ('지', 'EC'),
 ('않', 'VX'),
 ('고', 'EC'),
 ('개', 'NNG'),
 ('에', 'JKB'),
 ('도', 'JX'),
 ('나타나', 'VV'),
 ('ㄴ다고', 'EC'),
 ('지적', 'NNG'),
 ('하', 'XSV'),
 ('었', 'EP'),
 ('다', 'EF'),
 ('.', 'SF')]

In [60]:
wsd

[{'begin': 0.0,
  'end': 0.0,
  'id': 0.0,
  'position': 1.0,
  'scode': '00',
  'text': '주민',
  'type': 'NNG',
  'weight': 0.0},
 {'begin': 1.0,
  'end': 1.0,
  'id': 1.0,
  'position': 7.0,
  'scode': '09',
  'text': '들',
  'type': 'XSN',
  'weight': 4.0},
 {'begin': 2.0,
  'end': 2.0,
  'id': 2.0,
  'position': 10.0,
  'scode': '00',
  'text': '은',
  'type': 'JX',
  'weight': 1.0},
 {'begin': 3.0,
  'end': 3.0,
  'id': 3.0,
  'position': 14.0,
  'scode': '02',
  'text': '공장',
  'type': 'NNG',
  'weight': 5.4},
 {'begin': 4.0,
  'end': 4.0,
  'id': 4.0,
  'position': 20.0,
  'scode': '00',
  'text': '에서',
  'type': 'JKB',
  'weight': 1.0},
 {'begin': 5.0,
  'end': 6.0,
  'id': 5.0,
  'position': 27.0,
  'scode': '02',
  'text': '배출되',
  'type': 'VV',
  'weight': 3.2},
 {'begin': 7.0,
  'end': 7.0,
  'id': 6.0,
  'position': 33.0,
  'scode': '00',
  'text': 'ㄴ',
  'type': 'ETM',
  'weight': 1.0},
 {'begin': 8.0,
  'end': 8.0,
  'id': 7.0,
  'position': 37.0,
  'scode': '09',
  'text':

[('주민', 'NNG'),
 ('들', 'XSN'),
 ('은', 'JX'),
 ('공장', 'NNG'),
 ('에서', 'JKB'),
 ('배출되', 'VV'),
 ('ㄴ', 'ETM'),
 ('연기', 'NNG'),
 ('가', 'JKS'),
 ("'", 'SS'),
 ('남성', 'NNG'),
 ('이', 'JKS'),
 ('매우', 'MAG'),
 ('건강하', 'VA'),
 ('어', 'EC'),
 ('지', 'VX'),
 ('는', 'ETM'),
 ('부작용', 'NNG'),
 ("'", 'SS'),
 ('을', 'JKO'),
 ('일으키', 'VV'),
 ('ㄴ', 'EC'),
 ('다며', 'EC'),
 (',', 'SP'),
 ('이러하', 'VA'),
 ('ㄴ', 'ETM'),
 ('부작용', 'NNG'),
 ('은', 'JX'),
 ('사람', 'NNG'),
 ('에', 'JKB'),
 ('그치', 'VV'),
 ('지', 'EC'),
 ('않', 'VX'),
 ('고', 'EC'),
 ('개', 'NNG'),
 ('에', 'JKB'),
 ('도', 'JX'),
 ('나타나', 'VV'),
 ('ㄴ다고', 'EC'),
 ('지적하', 'VV'),
 ('었', 'EP'),
 ('다', 'EF'),
 ('.', 'SF')]

In [68]:
srl

[{'argument': [{'text': '공장',
    'type': 'ARG2',
    'weight': 0.120684,
    'word_id': 1.0},
   {'text': '연기', 'type': 'ARG1', 'weight': 0.274273, 'word_id': 3.0}],
  'sense': 1.0,
  'verb': '배출',
  'weight': 0.197479,
  'word_id': 2.0},
 {'argument': [{'text': "'남성",
    'type': 'ARG1',
    'weight': 0.142192,
    'word_id': 4.0},
   {'text': '매우', 'type': 'ARGM-EXT', 'weight': 0.48364, 'word_id': 5.0}],
  'sense': 1.0,
  'verb': '건강',
  'weight': 0.312916,
  'word_id': 6.0},
 {'argument': [{'text': '주민들',
    'type': 'ARG0',
    'weight': 0.326694,
    'word_id': 0.0},
   {'text': '연기', 'type': 'ARG0', 'weight': 0.381284, 'word_id': 3.0},
   {'text': "'남성이 매우 건강해지는 부작용'",
    'type': 'ARG1',
    'weight': 0.482167,
    'word_id': 7.0}],
  'sense': 1.0,
  'verb': '일으키',
  'weight': 0.396715,
  'word_id': 8.0},
 {'argument': [{'text': '이러한 부작용',
    'type': 'ARG1',
    'weight': 0.524875,
    'word_id': 10.0}],
  'sense': 1.0,
  'verb': '이러하',
  'weight': 0.524875,
  'word_id': 9.0},

In [29]:
twitter = Twitter()

In [35]:
tokens_ko = twitter.morphs(x['mainText'])

In [36]:
ko = nltk.Text(tokens_ko,name = x['title'])

In [39]:
print (len(ko.tokens))
print (len(set(ko.tokens)))
ko.vocab()

271
165


FreqDist({"'": 4,
          '*': 1,
          ',': 2,
          '.': 8,
          '20': 1,
          '=': 1,
          'blondie@segye.com': 1,
          '‘': 4,
          '“': 5,
          '”': 5,
          '가': 7,
          '가짜': 1,
          '간호사': 1,
          '강아지': 1,
          '개': 1,
          '건강': 1,
          '건강해': 1,
          '것': 2,
          '고': 5,
          '곳': 1,
          '공장': 6,
          '과': 1,
          '규제': 1,
          '그치': 1,
          '긍정': 1,
          '기자': 1,
          '까지': 1,
          '꾸며': 1,
          '나오고': 1,
          '나타난': 2,
          '남성': 4,
          '낸': 1,
          '년': 1,
          '느낄': 1,
          '는': 3,
          '는데': 1,
          '는지': 1,
          '늘었': 1,
          '다': 10,
          '다고': 2,
          '다며': 1,
          '대변인': 1,
          '덧붙였': 1,
          '되는': 1,
          '된': 1,
          '들': 5,
          '들어': 1,
          '들어선': 1,
          '듯': 1,
          '또는': 1,
          '라': 2,
          '라며': 1,
          

['주민',
 '들',
 '은',
 '공장',
 '에서',
 '배출',
 '된',
 '연기',
 '가',
 "'",
 '남성',
 '이',
 '매우',
 '건강해',
 '지는',
 '부작용',
 "'",
 '을',
 '일으킨',
 '다며',
 ',',
 '이러한',
 '부작용',
 '은',
 '사람',
 '에',
 '그치',
 '지',
 '않고',
 '개',
 '에도',
 '나타난',
 '다고',
 '지적했',
 '다',
 '.',
 '이어',
 '부작용',
 '으로',
 "'",
 '일상생활',
 '에',
 '어려움',
 '을',
 '느낄',
 '정도',
 "'",
 '라며',
 '역학',
 '조사',
 '가',
 '필요하다',
 '고',
 '덧붙였',
 '다',
 '.',
 '지역',
 '에서',
 '간호사',
 '로',
 '일하는',
 '한',
 '여성',
 '은',
 '“',
 '공장',
 '인근',
 '을',
 '산책',
 '하던',
 '중',
 '강아지',
 '가',
 '흥분',
 '상태',
 '인',
 '것',
 '을',
 '확인했',
 '다',
 '”',
 '며',
 '“',
 '공장',
 '이',
 '환경',
 '규제',
 '를',
 '준',
 '수',
 '하',
 '는지',
 '확인해야',
 '한',
 '다',
 '”',
 '고',
 '주장했',
 '다',
 '.',
 '공장',
 '은',
 '약',
 '20',
 '년',
 '전',
 '마을',
 '에',
 '들어선',
 '후',
 '현재',
 '까지',
 '치료',
 '제',
 '를',
 '생산했',
 '는데',
 '최근',
 '들어',
 '지역',
 '남성',
 '들',
 '에게',
 '예',
 '상치',
 '못',
 '한',
 '영향',
 '을',
 '미치',
 '고',
 '있다',
 '.',
 '시민',
 '들',
 '문제',
 '제기',
 '에',
 '공장',
 '대변인',
 '은',
 '“',
 '사랑',
 '의',
 '연기',
 '는',
 '존재하지',
 '않는',
 '다',
 